In [1]:
import pandas as pd
import re
df_t = pd.read_csv('transcript_all.csv')
df_p = pd.read_csv('product_all.csv')

In [2]:
def findnum(s):
    try:
        return re.findall(r'\d+', str(s))[0]
    except:
        return ''

df_t['tour_group_id']  = df_t.chat_start_url.apply(findnum)

tourid = df_p.loc[df_p.language_code == 'EN',['tour_group_id','name']]
tourid['tour_group_id'] = tourid.tour_group_id.apply(str)
df_m = pd.merge(df_t, tourid, on='tour_group_id')
df_text = df_m.groupby(['chat_id','name']).text.apply(' '.join).reset_index()

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=20)
t = tv.fit_transform(df_text.name.value_counts()[:100].index)
terms = pd.DataFrame({'term': tv.get_feature_names(), 'tfidf': t.toarray().sum(0)})
t.shape

/home/tianyi/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


(100, 513)

In [4]:
x = tv.transform(df_text.text)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df_text.name)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

In [31]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
sgd = SGDClassifier(random_state=100)
sgd_param = {'alpha': [0.005, 0.01, 0.015]
            }
clf = GridSearchCV(sgd, sgd_param, cv=5, n_jobs=-1, scoring='f1_weighted')

In [66]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb_param = {'alpha': [0.0015, 0.0016, 0.0018, 0.002]
            }
clf = GridSearchCV(nb, nb_param, cv=5, n_jobs=-1, scoring='f1_weighted')

In [67]:
%%time
clf.fit(x_train, y_train)

CPU times: user 136 ms, sys: 51.6 ms, total: 188 ms
Wall time: 780 ms


GridSearchCV(cv=5, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.0015, 0.0016, 0.0018, 0.002]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=0)

In [68]:
clf.cv_results_

{'mean_fit_time': array([0.05980678, 0.04901333, 0.03328414, 0.04040728]),
 'std_fit_time': array([0.02051127, 0.01095433, 0.00895858, 0.02583095]),
 'mean_score_time': array([0.01033731, 0.01531992, 0.00849371, 0.00889916]),
 'std_score_time': array([0.00255772, 0.0020584 , 0.00333761, 0.00436387]),
 'param_alpha': masked_array(data=[0.0015, 0.0016, 0.0018, 0.002],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.0015},
  {'alpha': 0.0016},
  {'alpha': 0.0018},
  {'alpha': 0.002}],
 'split0_test_score': array([0.17927972, 0.17954621, 0.17927143, 0.17891524]),
 'split1_test_score': array([0.27326351, 0.2733857 , 0.27329813, 0.27418948]),
 'split2_test_score': array([0.29498234, 0.29348613, 0.29153909, 0.29153909]),
 'split3_test_score': array([0.32186366, 0.32186366, 0.31761672, 0.32070729]),
 'split4_test_score': array([0.39777372, 0.39577786, 0.39577786, 0.39577786]),
 'mean_test_score': array([0.27551614, 0.

In [33]:
clf.cv_results_

{'mean_fit_time': array([0.63151245, 0.66161737, 0.61749902]),
 'std_fit_time': array([0.13954964, 0.14987718, 0.12486037]),
 'mean_score_time': array([0.00930529, 0.0081018 , 0.00620427]),
 'std_score_time': array([0.00201519, 0.00150126, 0.00118724]),
 'param_alpha': masked_array(data=[0.005, 0.01, 0.015],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.005}, {'alpha': 0.01}, {'alpha': 0.015}],
 'split0_test_score': array([0.17757634, 0.178976  , 0.17635944]),
 'split1_test_score': array([0.24934302, 0.25498104, 0.25390495]),
 'split2_test_score': array([0.31236104, 0.30576465, 0.31037164]),
 'split3_test_score': array([0.34034447, 0.32898634, 0.33611382]),
 'split4_test_score': array([0.39319814, 0.38531766, 0.40036665]),
 'mean_test_score': array([0.27534996, 0.2727925 , 0.27596748]),
 'std_test_score': array([0.07683354, 0.07211502, 0.07788572]),
 'rank_test_score': array([2, 3, 1], dtype=int32),
 'split0_train_

In [69]:
nb = MultinomialNB(alpha=0.0015)
nb.fit(x_train, y_train)

MultinomialNB(alpha=0.0015, class_prior=None, fit_prior=True)

In [71]:
f1_score(nb.predict(x_train), y_train, average = 'weighted')

0.496672809356436

In [72]:
f1_score(nb.predict(x_test), y_test, average = 'weighted')

0.3166453641515188

In [91]:
le.inverse_transform(nb.predict(tv.transform(['i have a ticket for the phatn of opera, but i change my plan, can i get refund'])))[0]

'The Phantom of the Opera'

In [94]:
nb.predict_proba(tv.transform(['can i chicago'])).max()

0.9221786541525197

In [95]:
nb.fit(x,y)

MultinomialNB(alpha=0.0015, class_prior=None, fit_prior=True)

In [100]:
import joblib
# save label encoder
filename = 'Product_Encoder_nb.sav'
joblib.dump(le, filename)

# save tfidf
filename = 'Product_Vectorizer_nb.sav'
joblib.dump(tv, filename)

# save rf model
filename = 'Product_Model_nb.sav'
joblib.dump(nb, filename)

['Product_Model_nb.sav']